In [22]:
import fourier
import math
import numpy as np
import random
import sys

from fractions import Fraction
from typing import List

from pyquil import Program
from pyquil.api import QVMConnection, WavefunctionSimulator
from pyquil.gates import MEASURE, I, X, CNOT, H, Z, RESET, CCNOT, PHASE
from pyquil.quil import address_qubits, DefGate
from pyquil.quilatom import Parameter, QubitPlaceholder, quil_exp

import time

qvm = QVMConnection()

def to_control(matrix):
    size = len(matrix)
    ret = []
    I = np.eye(size)
    for i in range(size):
        ret.append(np.concatenate([I[i], np.zeros(size)]))
    for i in range(size):
        ret.append(np.concatenate([np.zeros(size), matrix[i]]))
    return np.asarray(ret)

def bitfield(a, N):
    bits = [1 if digit=='1' else 0 for digit in bin(a)[2:]]
    prefix = [0] * (N - len(bits))
    return prefix + bits

def to_qubits(b, N):
    pq = Program()
    bits = bitfield(b, N)
    placeholders = []
    for i in bits:
        x = QubitPlaceholder()
        if i == 1:
            pq += X(x)
        else:
            pq += I(x)
        placeholders.append(x)
    return pq, placeholders

# https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/
def mod_inverse(a, m): 
    m0 = m 
    y = 0
    x = 1
  
    if (m == 1): 
        return 0
  
    while (a > 1 and m > 0): 
  
        # q is quotient 
        q = a // m 
  
        t = m 
  
        # m is remainder now, process 
        # same as Euclid's algo 
        m = a % m 
        a = t 
        t = y 
  
        # Update x and y 
        y = x - q * y 
        x = t 
  
  
    # Make x positive 
    if (x < 0) : 
        x = x + m0 
  
    return x

class ShorCircuit(object):
    def __init__(self, N):
        self.N = N
        # + 1 to prevent overflow in addition
        self.n = len([1 if digit=='1' else 0 for digit in bin(N)[2:]])
        self.add_ancilla = QubitPlaceholder()
        self.add_gate = lambda k: (lambda q: PHASE(math.pi / (2 ** (k - 1)), q))

    # Adding
    def add(self, b, a, c1=None, c2=None):
        pq = Program()
        a_bits = bitfield(a, len(b))
        for i in range(0, len(b)):
            for j in range(i, len(b)):
                if a_bits[j] == 1:
                    gate = self.add_gate(j - i + 1)(b[i])
                    if c1 is not None:
                        gate = gate.controlled(c1)
                        if c2 is not None:
                            gate = gate.controlled(c2)
                    pq += gate
        return pq

    def inv_add(self, b, a, c1=None, c2=None):
        pq = Program()
        a_bits = bitfield(a, len(b))
        for i in range(len(b) - 1, -1, -1):
            for j in range(len(b) - 1, i - 1, -1):
                if a_bits[j] == 1:
                    gate = self.add_gate(j - i + 1)(b[i]).dagger()
                    if c1 is not None:
                        gate = gate.controlled(c1)
                        if c2 is not None:
                            gate = gate.controlled(c2)
                    pq += gate 
        return pq

    # Quantum Fourier Transform
    def qft(self, b):
        pq = Program()
        for j in range(0, len(b)):
            pq += H(b[j])
            for k in range(j + 1, len(b)):
                diff = k - j
                pq += self.add_gate(diff + 1)(b[j]).controlled(b[k])
        return pq

    def inv_qft(self, b):
        pq = Program()
        for j in range(len(b) - 1, -1, -1):
            for k in range(len(b) - 1, j, -1):
                diff = k - j
                pq += self.add_gate(diff + 1)(b[j]).controlled(b[k]).dagger()
            pq += H(b[j])
        return pq

    # Add mod
    def add_mod(self, b, a, c1, c2):
        pq = Program()

        pq += self.add(b, a, c1, c2)
        pq += self.inv_add(b, self.N)
        pq += self.inv_qft(b)
        b_msb = b[0]
        pq += CNOT(b_msb, self.add_ancilla)
        pq += self.qft(b)
        
        pq += self.add(b, self.N, self.add_ancilla)
        pq += self.inv_add(b, a, c1, c2)
        
        pq += self.inv_qft(b)
        pq += X(b_msb)
        pq += CNOT(b_msb, self.add_ancilla)
        pq += X(b_msb)
        pq += self.qft(b)
        
        pq += self.add(b, a, c1, c2)
        return pq

    def inv_add_mod(self, b, a, c1, c2):
        pq = Program()
        pq += self.inv_add(b, a, c1, c2)

        pq += self.inv_qft(b)
        b_msb = b[0]
        pq += X(b_msb)
        pq += CNOT(b_msb, self.add_ancilla)
        pq += X(b_msb)
        pq += self.qft(b)

        pq += self.add(b, a, c1, c2)
        pq += self.inv_add(b, self.N, self.add_ancilla)

        pq += self.inv_qft(b)
        pq += CNOT(b_msb, self.add_ancilla)
        pq += self.qft(b)
        pq += self.add(b, self.N)
        pq += self.inv_add(b, a, c1, c2)
        return pq
    
    # b + ax mod N
    def cmult(self, b, a, c, x):
        pq = Program()

        pq += self.qft(b)
        a_bits = bitfield(a, self.N)[::-1]
        rev_x = x[::-1]
        for i in range(len(x)):
            a_mod = (2**i) * a % self.N
            pq += self.add_mod(b, a_mod, c, rev_x[i])
        pq += self.inv_qft(b)

        return pq
    
    def inv_cmult(self, b, a, c, x):
        pq = Program()
        
        pq += self.qft(b)
        a_bits = bitfield(a, self.N)[::-1]
        rev_x = x[::-1]
        for i in range(len(x) - 1, -1, -1):
            a_mod = (2**i) * a % self.N
            pq += self.inv_add_mod(b, a_mod, c, rev_x[i])
        pq += self.inv_qft(b)
        
        return pq
        
    def cswap(self, x, b, c):
        pq = Program()
        for i in range(len(x)):
            ind = -(i + 1)
            b_i = b[ind]
            x_i = x[ind] 
            pq += CNOT(b_i, x_i)
            pq += CCNOT(c, x_i, b_i)
            pq += CNOT(b_i, x_i)
        return pq

    def u_a(self, b_qubits, a, c, x):
        pq = Program()
        
        pq += self.cmult(b_qubits, a, c, x)
        pq += self.cswap(x, b_qubits, c)
        pq += self.inv_cmult(b_qubits, mod_inverse(a, self.N), c, x)
        return pq

    def partial_inv_qft(self, c, i, ro, total_iters):
        pq = Program()
        for k in range(total_iters - 1, i, -1):
            then_prog = Program(self.add_gate(k - i + 1)(c).dagger())
            pq.if_then(ro[k], then_prog)
        pq += H(c)
        return pq

    # Quantum Fourier Transform
    def end_qft(self, b):
        pq = Program()
        for j in range(0, len(b)):
            pq += H(b[j])
            for k in range(j + 1, len(b)):
                diff = k - j
                pq += self.add_gate(diff + 1)(b[j]).controlled(b[k])
        return pq
        
    # Builds the circuit for Shor's algorithm.
    def build(self, a):
        pq = Program()
        
        code, x_qubits = to_qubits(1, self.n)
        pq += code
        code, b_qubits = to_qubits(0, self.n + 1)
        pq += code
        
        total_iters = 2 * (self.n)
        ro = pq.declare('ro', 'BIT', total_iters + len(x_qubits))
        
        c = QubitPlaceholder()
        curr_a = a

        '''
        code, ctrl_qubits = to_qubits(0, total_iters)
        pq += code
        for i in range(total_iters):
            ind = total_iters - 1 - i
            c = ctrl_qubits[ind]
            pq += H(c)
            pq += self.u_a(b_qubits, curr_a, c, x_qubits)
            curr_a = curr_a ** 2 % self.N
        for i in range(len(x_qubits)):
            pq += MEASURE(x_qubits[i], ro[total_iters + i])
        ctrl_qubits = ctrl_qubits[::-1]
        pq += self.inv_qft(ctrl_qubits)
        #print(wave.wavefunction(address_qubits(pq)))
        for i in range(total_iters):
            pq += MEASURE(ctrl_qubits[i], ro[i])
        '''
        a_vals = []
        for i in range(total_iters):
            a_vals.append(curr_a)
            curr_a = pow(curr_a, 2, self.N)
        for ind in range(total_iters - 1, -1, -1):
            pq += H(c)
            pq += self.u_a(b_qubits, a_vals[ind], c, x_qubits)
            pq += self.partial_inv_qft(c, ind, ro, total_iters)
            pq += MEASURE(c, ro[ind])
            
            then_prog = Program(X(c))
            pq.if_then(ro[ind], then_prog)
            
        return address_qubits(pq)

def find_r(q, c, N):
    frac = Fraction(c, q)
    close_frac = frac.limit_denominator(N)
    return close_frac.denominator

def factor(N, y):
   # print("Factoring {}".format(N))
    if N % 2 == 0:
        #print("Factorization: ({}, {})".format(2, N // 2))
        return
    iters = 0
    circuit = ShorCircuit(N)
    q = 2 ** (2 * circuit.n)
        
    pq = circuit.build(y)
    result = qvm.run(pq)
    c = int(''.join(map(str, result[0])), 2)
    r = find_r(q, c, N) 
    #print("c: {}; r: {}".format(c, r))

In [23]:
def runShorAlgorithm(x, y) :
    timeBefore = time.perf_counter()
    factor(x, y)
    timeAfter = time.perf_counter()
    totalElapsedTime = timeAfter - timeBefore   
    formattedTotalElapsedTime = "{:.4f}".format(totalElapsedTime)      
    print(str(formattedTotalElapsedTime).replace(".",","))

In [25]:
for x in range(1, 43):
    runShorAlgorithm(15, 8)

1,9441
1,6117
1,6397
1,7255
1,7308
1,6578
1,7292
1,5935
1,5313
1,5735
1,6257
1,7696
1,5737
1,5137
1,5356
1,5825
1,5095
1,8578
1,6804
1,5921
1,4778
1,5337
1,5777
1,5327
1,5137
1,7475
2,1285
1,8898
2,2015
2,2053
1,8582
1,9287
1,5654
1,7142
1,7957
1,6660
1,5613
1,6910
1,7615
1,7909
1,5610
1,7149
